# Sesión 10: Word2Vec

---

Viernes 27 de Octubre de 2023

## Funcionamiento interno de convertir palabras a vectores utilizando skip-gram's

In [1]:
import io
import re
import string
import tqdm # for progress bar

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [3]:
SEED = 42   # Declaramos una semilla para que los resultados sean reproducibles
AUTOTUNE = tf.data.AUTOTUNE # AUTOTUNE nos permite que TensorFlow seleccione automáticamente el número de hilos de procesamiento que se utilizarán para ejecutar la operación

### Tokenización

In [5]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split()) # Convertimos la frase en una lista de tokens
print(len(tokens), tokens)

8 ['the', 'wide', 'road', 'shimmered', 'in', 'the', 'hot', 'sun']


In [7]:
# Creamos un diccionario de vocabulario
vocab, index = {}, 1 # Inicializamos el índice en 1 en lugar de 0
vocab['pad'] = 0 # Agregamos el token de relleno (padding)

for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1

vocab_size = len(vocab)
print(vocab)

{'pad': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [8]:
# Creamos un diccionario inverso, que nos permitirá convertir los índices en palabras
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: 'pad', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [9]:
# Reconstruimos la frase a partir de los índices (para comprobar que todo funciona correctamente)
example_sequence = [vocab[word] for word in tokens]
print(example_sequence) # Oración convertida en una secuencia de índices

[1, 2, 3, 4, 5, 1, 6, 7]


### Generación de skip-grams

In [10]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence, 
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0) # negative_samples=0 significa que no se muestrean ejemplos negativos

print(len(positive_skip_grams))

for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

26
(6, 7): (hot, sun)
(7, 6): (sun, hot)
(3, 1): (road, the)
(1, 3): (the, road)
(1, 5): (the, in)


### Generar muestras negativas

In [13]:
target_word, context_word = positive_skip_grams[0]
num_ns = 4 # Número de muestras negativas por cada ejemplo positivo

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))

negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes = context_class, # clase positiva
    num_true = 1, # número de clases positivas
    num_sampled = num_ns, # número de muestras negativas
    unique = True, # todas las muestras negativas deben ser únicas
    range_max = vocab_size, # rango de valores enteros
    seed = SEED, # semilla para reproducibilidad
    name = "negative_sampling" # nombre de la operación
)

print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([3 6 0 1], shape=(4,), dtype=int64)
['road', 'hot', 'pad', 'the']


Funcionamiento interno de la tokenización y generación de muestras negativas

In [14]:
# Concatenación de muestras positivas y negativas

squeezed_context_class = tf.squeeze(context_class, 1) # Eliminamos las dimensiones de tamaño 1 de la forma de un tensor
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0) # Concatenamos las muestras positivas y negativas
label = tf.constant([1] + [0]*num_ns, dtype="int64") # Creamos las etiquetas correspondientes
target = target_word # Definimos el objetivo

print(f"target_index    : {target}") # Índice de la palabra objetivo
print(f"target_word     : {inverse_vocab[target_word]}") # Palabra objetivo
print(f"context_indices : {context}") # Índices de las palabras del contexto
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}") # Palabras del contexto
print(f"label           : {label}") # Etiqueta

target_index    : 6
target_word     : hot
context_indices : [7 3 6 0 1]
context_words   : ['sun', 'road', 'hot', 'pad', 'the']
label           : [1 0 0 0 0]


---

## Implementación de la red neuronal Word2Vec

### Generación de datos de entrenamiento

In [ ]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  targets, contexts, labels = [], [], []

  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size) # Creamos una tabla de muestreo para muestrear palabras del vocabulario

  for sequence in tqdm.tqdm(sequences): # Iteramos sobre las secuencias

    # Generación de skip-grams positivos
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
        sequence, 
        vocabulary_size=vocab_size,
        sampling_table=sampling_table,
        window_size=window_size,
        negative_samples=0)
    
    # Generación de muestras negativas
    for target_word, context_word in positive_skip_grams:

      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1, 
          num_sampled=num_ns, 
          unique=True, 
          range_max=vocab_size, 
          seed=seed, 
          name="negative_sampling")
      
      # Concatenaciones y etiquetas
      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")
      

			# Añadimos los datos a las listas
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)
      
  return targets, contexts, labels

### Descargar documento (Text Corpus)

In [23]:
path_to_file = keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [24]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool)) # Cargamos el dataset

### Estandarización

In [33]:
def custom_standardization(input_data): # Función para eliminar los signos de puntuación
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

vocab_size = 4096
sequence_length = 10

In [34]:
vacab_size = 4096 # Tamaño del vocabulario
sequence_length = 10 # Longitud de cada oración

vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardization, # Función de estandarización
    max_tokens=vocab_size, # Tamaño del vocabulario
    output_mode='int', # Modo de salida
    output_sequence_length=sequence_length) # Longitud de cada secuencia

vectorize_layer.adapt(text_ds.batch(1024)) # Adaptamos el vectorizador al dataset

In [35]:
inverse_vocab = vectorize_layer.get_vocabulary() # Diccionario inverso
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [36]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch() # Vectorizamos el dataset

In [37]:
sequences = list(text_vector_ds.as_numpy_iterator()) # Convertimos el dataset en una lista de secuencias
print(len(sequences))

for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

32777
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
